# Solving for RO Membrane Parameters Using a Module Datasheet
In this demonstration, we will build a simple RO flowsheet and determine the membrane water permeability and salt permeability coefficients using an example module datasheet.

### Example RO membrane specifications sheet

Membrane manufacturers usually provide specifications on membrane modules but do not always provide the membrane permeability coefficients.

We can use WaterTAP to determine the permeability coefficients using the module test data reported in the module datasheet

<p style="text-align: center"><img src="graphics/BW30-typical_properties.png" height="300">

## Part 1: Example RO flowsheet

<p style="text-align: center"><img src="graphics/RO_Stage.png" width="50%">

## Build simple RO model

In this section we will build an RO 0D model with assumed membrane and module parameters

### Import necessary libraries

In [ ]:
from pyomo.environ import (
    check_optimal_termination,
    ConcreteModel,
    value,
    Var,
    assert_optimal_termination,
    units as pyunits,
)

from idaes.core import FlowsheetBlock
from watertap.property_models.NaCl_prop_pack import NaClParameterBlock
from idaes.core.util.scaling import calculate_scaling_factors, set_scaling_factor

from watertap.unit_models.reverse_osmosis_0D import (
    ReverseOsmosis0D,
    ConcentrationPolarizationType,
    MassTransferCoefficient,
    PressureChangeType,
)
from watertap.core.util.model_diagnostics.infeasible import *
from watertap.core.util.initialization import *
from idaes.core.util.model_statistics import degrees_of_freedom
from watertap.core.solvers import get_solver

### Create a flowsheet block and RO model

In [ ]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = NaClParameterBlock()

m.fs.RO = ReverseOsmosis0D(
    property_package=m.fs.properties,
    has_pressure_change=True,
    pressure_change_type=PressureChangeType.calculated,
    mass_transfer_coefficient=MassTransferCoefficient.calculated,
    concentration_polarization_type=ConcentrationPolarizationType.calculated,
    has_full_reporting=True,
)

### Specify values for system variables

In [ ]:
# Fix the 4 inlet state variables
m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "H2O"].fix(0.965)  # Feed water flow (kg/s)
m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "NaCl"].fix(0.035)  # Feed salt flow (kg/s)
m.fs.RO.inlet.pressure[0].fix(50e5)  # Feed pressure (Pa)
m.fs.RO.inlet.temperature[0].fix(298)  # Feed temperature (K)

# Fix 2 membrane properties
m.fs.RO.A_comp.fix(4.2e-12)  # membrane water permeability coeff (m/Pa/s)
m.fs.RO.B_comp.fix(3.5e-8)  # membrane salt permeability coeff (m/s)

# Fix 4 module specifications
m.fs.RO.area.fix(50)  # Membrane stage area (m^2)
m.fs.RO.width.fix(5)  # Membrane stage width (m)
m.fs.RO.feed_side.channel_height.fix(1e-3)  # Channel height in membrane stage (m)
m.fs.RO.feed_side.spacer_porosity.fix(0.97)  # Spacer porosity in membrane stage (-)

# 1 outlet state variable
m.fs.RO.permeate.pressure[0].fix(101325)  # Permeate pressure (Pa)

### Check degrees of freesom

In [ ]:
print("Degrees of Freedom:", degrees_of_freedom(m))

### Scale all variables

In [ ]:
# Set scaling factors for component mass flowrates.
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1e2, index=("Liq", "NaCl"))

# Set scaling factor for membrane area.
set_scaling_factor(m.fs.RO.area, 1e-2)

# Calculate scaling factors for all other variables.
calculate_scaling_factors(m)

### Initialize and solve the model

In [ ]:
m.fs.RO.initialize()

# Solve the RO Unit
solver = get_solver()
results = solver.solve(m)

assert_optimal_termination(results)

## 1.2: Calculate the water permeability coefficient (A_comp)

### Using permeate flow rate

In this section we will unfix and calculate the water permeability coefficient in the model by fixing the permeate flux

In [ ]:
# We can start by unfixing the water permeability coefficient
m.fs.RO.A_comp.unfix()
print("Degrees of Freedom:", degrees_of_freedom(m.fs.RO))

In [ ]:
# And then we can define a fixed value for the permeate flowrate
m.fs.RO.mixed_permeate[0.0].flow_mass_phase_comp["Liq", "H2O"].fix(0.4)
print("Degrees of Freedom:", degrees_of_freedom(m.fs.RO))

In [ ]:
# Initialize and solve the model given the new fixed value
m.fs.RO.initialize()
results = solver.solve(m)
assert_optimal_termination(results)

m.fs.RO.A_comp.display()

#### Using water recovery

In this section we will calculate the water permeability coefficient in the model by fixing the water recovery

In [ ]:
# Let's unfix the permeate flowrate that was fixed earlier
m.fs.RO.mixed_permeate[0.0].flow_mass_phase_comp["Liq", "H2O"].unfix()

In [ ]:
# Similar to the previous example, we can fix the recovery to 40%
m.fs.RO.recovery_vol_phase[0.0, "Liq"].fix(0.4)

print("Degrees of Freedom:", degrees_of_freedom(m.fs.RO))

In [ ]:
results = solver.solve(m)
assert_optimal_termination(results)

m.fs.RO.A_comp.display()

## Part 2: Estimating membrane properties given a RO module datasheet

<p style="text-align: center"><img src="graphics/BW30-element_dimensions.png" height="250">
<p style="text-align: center"><img src="graphics/BW30-typical_properties.png" height="250">

Lets translate the information in the spec sheet to inputs for the RO model

In [ ]:
# Constant
density = 997 * pyunits.kg / pyunits.m**3

# Membrane area, length and channel height
mem_area = 41 * pyunits.m**2
length = 1.016 * pyunits.m
channel_height = 0.7112 * 1e-3 * pyunits.m  # Channel height 28 mil to m

# Given permeate flow rate
permeate_vol_flow_rate = pyunits.convert(
    48 * pyunits.m**3 / pyunits.day, to_units=pyunits.m**3 / pyunits.s
)
permeate_mass_flow_rate = permeate_vol_flow_rate * density

# Salt rejection
rejection = 0.997  # Dimensionless

# Recovery
recovery = 0.15  # Dimensionless

# Feed concentration
feed_conc = 2000 * pyunits.mg / pyunits.L

# Inlet Pressure
feed_pressure = 15.5 * pyunits.bar

# Calculate feed flow rate and concentration using the recovery
feed_vol_flow_rate = permeate_vol_flow_rate / recovery  # m^3/s
feed_mass_flow_rate = feed_vol_flow_rate * density  # kg/s
feed_salt_flow_rate = feed_conc * feed_vol_flow_rate

print(
    "Feed mass flow rate:",
    value(feed_mass_flow_rate),
    pyunits.get_units(feed_mass_flow_rate),
)
print(
    "Feed salt mass flow:",
    value(feed_salt_flow_rate),
    pyunits.get_units(feed_salt_flow_rate),
)

### 2.1 Build the RO

In this section, we will setup the RO model to reflect RO module datasheet.

We will start with example A_comp and B_comp values

In [ ]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = NaClParameterBlock()

m.fs.RO = ReverseOsmosis0D(
    property_package=m.fs.properties,
    has_pressure_change=True,
    pressure_change_type=PressureChangeType.calculated,
    mass_transfer_coefficient=MassTransferCoefficient.calculated,
    concentration_polarization_type=ConcentrationPolarizationType.calculated,
    has_full_reporting=True,
)

# Fix the 4 inlet state variables
m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "H2O"].fix(
    feed_mass_flow_rate
)  # Feed water flow (kg/s)
m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "NaCl"].fix(
    feed_salt_flow_rate
)  # Feed salt flow (kg/s)

m.fs.RO.inlet.pressure[0].fix(feed_pressure)  # Feed pressure (Pa)
m.fs.RO.inlet.temperature[0].fix(298)  # Feed temperature (K)

# Fix 2 membrane properties
m.fs.RO.A_comp.fix(4.2e-12)  # membrane water permeability coeff (m/Pa/s)
m.fs.RO.B_comp.fix(3.5e-8)  # membrane salt permeability coeff (m/s)

# Fix 4 module specifications
m.fs.RO.area.fix(mem_area)  # Membrane area (m^2)
m.fs.RO.length.fix(length)  # Membrane stage length (m)
m.fs.RO.feed_side.channel_height.fix(
    channel_height
)  # Channel height in membrane stage (m)
m.fs.RO.feed_side.spacer_porosity.fix(0.97)  # Spacer porosity in membrane stage (-)

# 1 outlet state variable
m.fs.RO.permeate.pressure[0].fix(101325)  # Permeate pressure (Pa)

# Set scaling factors for component mass flowrates.
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1e6, index=("Liq", "NaCl"))

# Release constraints related to low concentrations
for item in [m.fs.RO.permeate_side, m.fs.RO.feed_side.properties_interface]:
    for idx, param in item.items():
        param.molality_phase_comp["Liq", "NaCl"].setlb(1e-5)
        param.pressure_osm_phase["Liq"].setlb(10)

m.fs.RO.flux_mass_phase_comp.setub(0.05)

# Set scaling factor for membrane area.
set_scaling_factor(m.fs.RO.area, 1e-2)

# Calculate scaling factors for all other variables.
calculate_scaling_factors(m)

m.fs.RO.initialize()

# Solve the RO Unit
solver = get_solver()
results = solver.solve(m)
assert_optimal_termination(results)

<div class="alert alert-block alert-info">
<b>NOTE:</b> The salinity given in the datasheet is low (2g/L). 

Some of the variables in the NaCl property model are scaled and constrained to limits more relevant to seawater concentrations. 

Sometimes adjusting the bounds on these variables is required to solve in these different conditions
</div>

In [ ]:
print("Degrees of Freedom:", degrees_of_freedom(m.fs.RO))

### 2.2 Calculate the water permeability coefficient (A_comp)

In [ ]:
# We first unfix the water permeability coefficient
m.fs.RO.A_comp.unfix()

# And then we can define a fixed value for the permeate flowrate
m.fs.RO.mixed_permeate[0.0].flow_mass_phase_comp["Liq", "H2O"].fix(
    permeate_mass_flow_rate
)

print("DOF = ", degrees_of_freedom(m))
solver = get_solver()
results = solver.solve(m)

assert_optimal_termination(results)

m.fs.RO.A_comp.display()

### 2.3 Calculate the salt permeability coefficient (B_comp)

In this section we calculate the salt permeability coefficient using the salt rejection data provided to us

In [ ]:
# We first unfix the permeate water flowrate
m.fs.RO.mixed_permeate[0.0].flow_mass_phase_comp["Liq", "H2O"].unfix()

# Fix the water permeability coefficient and unfix the salt permeability coefficient
m.fs.RO.A_comp.fix()
m.fs.RO.B_comp.unfix()

# And then we can fix value for the salt rejection
m.fs.RO.rejection_phase_comp[0, "Liq", "NaCl"].fix(0.997)

print("DOF = ", degrees_of_freedom(m))
solver = get_solver()
results = solver.solve(m)

assert_optimal_termination(results)

m.fs.RO.B_comp.display()

# Try it yourself

## Solve for Spacer Porosity

We will be using the A and B values we calculated earlier to determine the spacer porosity given the operating limits of the membrane shown in the screen shot below.

<p style="text-align: center"><img src="graphics/BW30_2.png" width="70%">

We have updated values for:
1. Feed pressure = Maximum operating pressure
2. Feed flow rate = Maximum feed flow

We can now fix the pressure drop and calculate the spacer porosity

### Datasheet inputs

In [ ]:
feed_mass_flow_rate = pyunits.convert(
    19 * pyunits.m**3 / pyunits.hr * density, to_units=pyunits.kg / pyunits.s
)  # Feed mass flowrate in kg/s

feed_pressure = 41 * pyunits.bar
pressure_loss = -1 * pyunits.bar

In [ ]:
# Here we are unfixing the permeate flowrate and fixing the salt rejection
m.fs.RO.mixed_permeate[0.0].flow_mass_phase_comp["Liq", "H2O"].unfix()
m.fs.RO.rejection_phase_comp[0, "Liq", "NaCl"].unfix()

# Here we are fixing both membrane parameters after fitting
m.fs.RO.A_comp.fix()
m.fs.RO.B_comp.fix()

<details>
  <summary>Click the arrow for hint #1!</summary>
    
To update the feed flow rate

`m.fs.RO.inlet.flow_mass_phase_comp[0, "Liq", "H2O"].fix(feed_mass_flow_rate)`
</details>

<details>
  <summary>Click the arrow for hint #2!</summary>

To update the feed pressure 

`m.fs.RO.inlet.pressure[0].fix(feed_pressure)`
</details>

<details>
  <summary>Click the arrow for hint #3!</summary>

To unfix the spacer porosity
   
`m.fs.RO.feed_side.spacer_porosity.unfix()`
</details>

<details>
  <summary>Click the arrow for hint #4!</summary>

To fix the pressure drop here in bar
   
`m.fs.RO.deltaP.fix(pressure_loss)`
</details>


In [ ]:
# Update the feed flow rate here in kg/s - Hint 1


# Update the feed pressure here in bar - Hint 2


# Unfix the spacer porosity - Hint 3


# Fix the pressure drop here in bar - Hint 4


# Check the degrees of freedom
print("DOF = ", degrees_of_freedom(m))

results = solver.solve(m)
assert_optimal_termination(results)

# Print the spacer porosity value
print("Fitted spacer porosity:", value(m.fs.RO.feed_side.spacer_porosity))